# Dynamic Pricing for Urban Parking Lots
**Capstone Project - Summer Analytics 2025**

This notebook implements a dynamic pricing engine for urban parking lots using:
- Linear occupancy-based model
- Multi-feature demand-based model
- Competitive pricing model with spatial awareness
- Real-time streaming simulation using Pathway
- Interactive visualization using Bokeh


In [ ]:
!pip install bokeh pathway

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
import pathway as pw

output_notebook()


In [ ]:

# Load dataset
df = pd.read_csv("dataset.csv")
df.fillna(0, inplace=True)

# Feature Engineering
df['Occupancy_rate'] = df['Occupancy'] / df['Capacity']
df['VehicleType_encoded'] = df['VehicleType'].map({'car': 1, 'bike': 0.5, 'truck': 1.5})


In [ ]:

def model1_linear(df, alpha=2.0, base=10):
    df['price_model1'] = base + alpha * df['Occupancy_rate']
    return df

df = model1_linear(df)


In [ ]:

def model2_demand(df, base=10):
    a, b, c, d, e = 1.2, 0.5, 0.3, 1.0, 0.8
    demand = (
        a * df['Occupancy_rate'] + b * df['QueueLength'] -
        c * df['TrafficConditionNearby'] + d * df['IsSpecialDay'] +
        e * df['VehicleType_encoded']
    )
    df['norm_demand'] = (demand - demand.min()) / (demand.max() - demand.min())
    df['price_model2'] = base * (1 + 1.0 * df['norm_demand'])
    df['price_model2'] = df['price_model2'].clip(lower=base*0.5, upper=base*2.0)
    return df

df = model2_demand(df)


In [ ]:

def get_nearby_lots(df_time, ID, radius=0.01):
    info = df_time[df_time['ID'] == ID][['Latitude', 'Longitude']].iloc[0]
    all_locations = df_time[['ID', 'Latitude', 'Longitude']].drop_duplicates()
    all_locations['dist'] = ((all_locations['Latitude'] - info['Latitude'])**2 +
                             (all_locations['Longitude'] - info['Longitude'])**2)**0.5
    return all_locations[(all_locations['dist'] < radius) & (all_locations['ID'] != ID)]['ID'].tolist()

def model3_competitive(df, base=10):
    df['price_model3'] = df['price_model2']
    for time in df['timestamp'].unique():
        temp = df[df['timestamp'] == time]
        for lot in df['ID'].unique():
            occ = temp[temp['ID'] == lot]['Occupancy'].values[0]
            cap = temp[temp['ID'] == lot]['Capacity'].values[0]
            p2 = temp[temp['ID'] == lot]['price_model2'].values[0]
            nearby = get_nearby_lots(temp, lot)
            competitors = temp[temp['ID'].isin(nearby)]['price_model2']
            if len(competitors) > 0:
                avg_comp = competitors.mean()
                if occ >= cap and p2 > avg_comp:
                    df.loc[(df['ID'] == lot) & (df['timestamp'] == time), 'price_model3'] = avg_comp * 0.95
                elif p2 < avg_comp:
                    df.loc[(df['ID'] == lot) & (df['timestamp'] == time), 'price_model3'] = avg_comp * 0.98
    return df

df = model3_competitive(df)


In [ ]:

sample_lot = df[df['ID'] == df['ID'].unique()[0]]
sample_lot['timestamp'] = sample_lot['timestamp'].astype(str)

source = ColumnDataSource(data={
    'timestamp': sample_lot['timestamp'],
    'price1': sample_lot['price_model1'],
    'price2': sample_lot['price_model2'],
    'price3': sample_lot['price_model3']
})

p = figure(title="Pricing Comparison - Lot 1", x_range=sample_lot['timestamp'],
           x_axis_label='Timestamp', y_axis_label='Price ($)', plot_width=900, plot_height=400)

p.line(x='timestamp', y='price1', source=source, line_width=2, color='blue', legend_label='Model 1')
p.line(x='timestamp', y='price2', source=source, line_width=2, color='green', legend_label='Model 2')
p.line(x='timestamp', y='price3', source=source, line_width=2, color='red', legend_label='Model 3')

p.add_tools(HoverTool(tooltips=[
    ("Time", "@timestamp"),
    ("Model 1", "@price1{$0.00}"),
    ("Model 2", "@price2{$0.00}"),
    ("Model 3", "@price3{$0.00}")
]))

p.legend.location = "top_left"
p.xaxis.major_label_orientation = 0.8
show(p)


In [ ]:

class Schema(pw.io.csv.Schema):
    ID: str
    timestamp: str
    Latitude: float
    Longitude: float
    Capacity: int
    Occupancy: int
    QueueLength: int
    VehicleType: str
    TrafficConditionNearby: float
    IsSpecialDay: int

def pathway_price(row):
    occ_rate = row.Occupancy / row.Capacity if row.Capacity else 0
    weight = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}.get(row.VehicleType, 1.0)
    demand = 1.2 * occ_rate + 0.5 * row.QueueLength - 0.3 * row.TrafficConditionNearby + row.IsSpecialDay + 0.8 * weight
    norm = demand / 10
    return max(5, min(20, 10 * (1 + norm)))

class App:
    def __init__(self):
        self.input = pw.io.csv.read("dataset.csv", schema=Schema, mode="streaming")
        self.output = self.input.select(
            ID=self.input.ID,
            timestamp=self.input.timestamp,
            price=pw.apply(pathway_price, self.input)
        )
        self.output.show()

# To run:
# app = App()
# pw.run()
